In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.simplefilter(action="ignore")
import time
import pandas as pd

In [12]:
driver = webdriver.Chrome()
url = "https://sg.sbiz.or.kr/godo/index.sg"
driver.get(url)

In [13]:
df = pd.read_csv('data/맥도날드_결측치.csv', encoding='utf-8')
address_list = (df['address']).tolist()
address_list

['경기 수원시 영통구 창룡대로 370',
 '경기 평택시 경기대로 570',
 '전남 목포시 통일대로 88',
 '전북 전주시 덕진구 백제대로 692']

In [14]:
def input_address(driver, address):
    
    #검색 창에 주소 입력
    search_box = driver.find_element(By.CSS_SELECTOR, "#searchAddress")
    search_box.clear()
    search_box.send_keys(address)

    #검색 버튼 클릭
    time.sleep(0.2)
    element = driver.find_element(By.CSS_SELECTOR, "#layerPopAddressMove")
    driver.execute_script("arguments[0].click();", element)

    #첫 번째 검색 결과 클릭
    time.sleep(0.2)
    address_list_element = driver.find_element(By.CSS_SELECTOR, "#adrsList > li > label")
    driver.execute_script("arguments[0].click();", address_list_element)

    #다음 버튼
    time.sleep(0.2)
    next_button = driver.find_element(By.CSS_SELECTOR, "#container > div:nth-child(1) > div:nth-child(3) > div.foot > a:nth-child(2) > span")
    driver.execute_script("arguments[0].click();", next_button)

def zoom_in_out(driver):
    
    #가끔 검색이 제대로 되지 않음
    zoom_in = driver.find_element(By.CSS_SELECTOR, "#container > div.custom_zoomcontrol > div:nth-child(3) > a:nth-child(2)")
    zoom_out = driver.find_element(By.CSS_SELECTOR, "#container > div.custom_zoomcontrol > div:nth-child(3) > a:nth-child(1)")
    driver.execute_script("arguments[0].click();", zoom_in)
    driver.execute_script("arguments[0].click();", zoom_out)

def select_map_options(driver):
    
    #맵 포인터 클릭
    time.sleep(0.2)
    sanggwon_button = driver.find_element(By.CSS_SELECTOR, "#map > div:nth-child(1) > div > div:nth-child(6) > div:nth-child(2) > div > ul > li.child > label")
    sanggwon_button.click()

    #반경 내 상권 정보 열람
    time.sleep(0.2)
    bankyung_button = driver.find_element(By.CSS_SELECTOR, "#map > div:nth-child(1) > div > div:nth-child(6) > div:nth-child(2) > div > ul > li.child > div > ul > li:nth-child(2) > label")
    bankyung_button.click()

    #반경 거리 700m
    time.sleep(0.2)
    bankyung = driver.find_element(By.CSS_SELECTOR, "#auto_circle > div > div.midd > ul > li:nth-child(7) > label")
    bankyung.click()

    #확인 버튼
    time.sleep(0.2)
    element = driver.find_element(By.CSS_SELECTOR, "#auto_circle > div > div.foot > a:nth-child(2) > span")
    element.click()

    #분석 버튼
    time.sleep(0.2)
    analyze_button = driver.find_element(By.CSS_SELECTOR, "#map > div:nth-child(1) > div > div:nth-child(6) > div:nth-child(3) > img")
    analyze_button.click()

def analyze_sales_data(driver):
    sales_data = []

    # 첫 페이지에서 이동하여 데이터 수집 (Upso : 반경 내 업소 수, SalesAmt : 월 평균 매출액)
    some_tag1 = driver.find_element(By.CSS_SELECTOR, "#page1 > div.report-pop-layer > span:nth-child(3)")
    action = ActionChains(driver)
    action.move_to_element(some_tag1).perform()

    time.sleep(0.2)
    Upso = driver.find_element(By.CSS_SELECTOR, "#page1 > div.report-pop-layer > div.analysis-section.analysis-01 > div.analysis-content > div:nth-child(1) > div > div > span").text
    sales_data.append(Upso)
    SalesAmt = driver.find_element(By.CSS_SELECTOR, "#salesSmryInfoCurSaleAmt").text
    sales_data.append(SalesAmt)

    # 두 번째 페이지로 이동하여 데이터 수집 (FlowPop : 유동인구, EmpPop : 직장인구, AbodePop : 주거인구, EmpAvgCo : 주거인구 평균소득)
    time.sleep(0.2)
    some_tag2 = driver.find_element(By.CSS_SELECTOR, "#page1 > div.report-pop-layer > span:nth-child(5)")
    action.move_to_element(some_tag2).perform()

    time.sleep(0.2)
    FlowPop = driver.find_element(By.CSS_SELECTOR, "#flowPopSmryInfoFlowPop").text
    sales_data.append(FlowPop)
    EmpPop = driver.find_element(By.CSS_SELECTOR, "#empAbodePopSmryInfoEmpPop").text
    sales_data.append(EmpPop)
    AbodePop = driver.find_element(By.CSS_SELECTOR, "#empAbodePopSmryInfoAbodePop").text
    sales_data.append(AbodePop)
    EmpAvgCo = driver.find_element(By.CSS_SELECTOR, "#empAbodePopSmryInfoEmpAvgCo").text
    sales_data.append(EmpAvgCo)

def close_report(driver):

    #창 닫기
    time.sleep(0.2)
    driver.find_element(By.XPATH ,'//*[@id="bind"]/div[1]/a').click()

In [15]:
cnt = 0
data_list=[]

for address in address_list:
    try:
        input_address(driver, address)
        zoom_in_out(driver)
        select_map_options(driver)

        time.sleep(10) # 웹페이지 대기 시간

        sales_data = []

        # 첫 페이지에서 이동하여 데이터 수집 (Upso : 반경 내 업소 수, SalesAmt : 월 평균 매출액)
        some_tag1 = driver.find_element(By.CSS_SELECTOR, "#page1 > div.report-pop-layer > span:nth-child(3)")
        action = ActionChains(driver)
        action.move_to_element(some_tag1).perform()

        time.sleep(0.2)
        Upso = driver.find_element(By.CSS_SELECTOR, "#page1 > div.report-pop-layer > div.analysis-section.analysis-01 > div.analysis-content > div:nth-child(1) > div > div > span").text
        sales_data.append(Upso)
        SalesAmt = driver.find_element(By.CSS_SELECTOR, "#salesSmryInfoCurSaleAmt").text
        sales_data.append(SalesAmt)

        # 두 번째 페이지로 이동하여 데이터 수집 (FlowPop : 유동인구, EmpPop : 직장인구, AbodePop : 주거인구, EmpAvgCo : 주거인구 평균소득)
        time.sleep(0.2)
        some_tag2 = driver.find_element(By.CSS_SELECTOR, "#page1 > div.report-pop-layer > span:nth-child(5)")
        action.move_to_element(some_tag2).perform()

        time.sleep(0.2)
        FlowPop = driver.find_element(By.CSS_SELECTOR, "#flowPopSmryInfoFlowPop").text
        sales_data.append(FlowPop)
        EmpPop = driver.find_element(By.CSS_SELECTOR, "#empAbodePopSmryInfoEmpPop").text
        sales_data.append(EmpPop)
        AbodePop = driver.find_element(By.CSS_SELECTOR, "#empAbodePopSmryInfoAbodePop").text
        sales_data.append(AbodePop)
        EmpAvgCo = driver.find_element(By.CSS_SELECTOR, "#empAbodePopSmryInfoEmpAvgCo").text
        sales_data.append(EmpAvgCo)
        
        close_report(driver)

        print(cnt, sales_data)
        data_list.append(sales_data)
        cnt += 1

    except Exception as e:
        print(f"예상매출 정보를 찾을 수 없습니다. 주소: {address}")
        # 오류가 발생하면 모든 판매 정보를 0으로 설정합니다.
        sales_data = [0] * 6
        data_list.append(sales_data)

0 ['11개', '2,593만원', '125,002명', '19,396명', '10,530명', '339만원']
1 ['2개', '15,528만원', '64,615명', '5,870명', '13,725명', '327만원']
2 ['2개', '1,860만원', '129,825명', '11,056명', '10,768명', '318만원']
3 ['1개', '0만원', '48,551명', '12,494명', '9,246명', '327만원']


In [16]:
df_data = pd.DataFrame(data_list)
df_data.columns = ['Upso', 'SalesAmt', 'FlowPop', 'EmpEop', 'AbodePop', 'EmpAvgCo']
df_result = pd.concat([df,df_data], axis=1)
df_result

,name,address,lat,lng,Upso,SalesAmt,FlowPop,EmpEop,AbodePop,EmpAvgCo
0,동수원GS DT,경기 수원시 영통구 창룡대로 370,37.294811,127.048409,11개,"2,593만원","125,002명","19,396명","10,530명",339만원
1,평택 세교DT점,경기 평택시 경기대로 570,37.005327,127.084502,2개,"15,528만원","64,615명","5,870명","13,725명",327만원
2,목포하당DT점,전남 목포시 통일대로 88,34.800431,126.433771,2개,"1,860만원","129,825명","11,056명","10,768명",318만원
3,전주인후DT점,전북 전주시 덕진구 백제대로 692,35.844009,127.145058,1개,0만원,"48,551명","12,494명","9,246명",327만원


In [17]:
df_result.to_csv('data/맥도날드_결측치.csv', index=False, encoding='utf-8-sig')